# Airline Assistant Project

Creating a chat assistant for a airline company

In [2]:
import os
import json
from openai import OpenAI
import gradio as gr

In [9]:
# Creating llama model and needed Constants
Ollama_url = "http://localhost:11434/v1"
ollama_api_key = "ollama"

llama_model = 'llama3.1'

ollama = OpenAI(base_url=Ollama_url, api_key=ollama_api_key)


In [58]:
system_message = "You are a helpful assistant for an Airline named Latios Airlines"
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so and do not make up anything."

In [20]:
def chat(message, history):
    messages = [{"role":"system", "content":system_message}] + history + [{"role":"user", "content":message}]

    response = ollama.chat.completions.create(model=llama_model, messages=messages)

    return response.choices[0].message.content

In [22]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.


In [65]:
# Create a function that returns ticket price from a dictionary

ticket_price = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool that gets ticket price for {destination_city}")
    if destination_city != None:
        city = destination_city.strip().lower()
    
    return ticket_price.get(city, "Unknown")

In [69]:
get_ticket_price("yes")

Tool that gets ticket price for yes


'Unknown'

In [63]:
# Tool -> Dictionary Structure that is required to describe the function 
# Give an example for the llm is a good to helpthe llm 
price_function = {
    "name" : "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters":{
        "type":"object",
        "properties" : {
            "destination_city": {
                "type": "string", 
                "description":"City the customer wants to travel to",
            },
        },
        "required": ["destination_ctiy"],
        "additionalProperties":False
    }
}

In [44]:
tools = [{"type":"function", "function": price_function}]

# Getting OpenAI to use the Tool

set up a function for the LLM to call the tool function I created

In [70]:
#unpacks the message from llm 
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role":"tool",
        "content": json.dumps({"destination_city":city, "price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city


In [71]:
def chat(message, history):
    messages = [{"role":"system", "content":system_message}] + history + [{"role":"user", "content":message}]

    response = ollama.chat.completions.create(model=llama_model, messages=messages, tools=tools, tool_choice="auto")
    #if response contains need for tool then
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        #unpack message from llm and return result
        response,city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)

        response = ollama.chat.completions.create(model=llama_model, messages=messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


Tool that gets ticket price for N/A
Tool that gets ticket price for Tokyo
Tool that gets ticket price for Hoen
Tool that gets ticket price for Hawaii
Tool that gets ticket price for Tokyo
Tool that gets ticket price for Tokyo
Tool that gets ticket price for Honolulu
Tool that gets ticket price for Berlin
